In [1]:
# Get the mapping of each id & associated store first

# upto 20200418


import pandas as pd
import numpy as np
import os
import datetime
import logging
import hashlib
import gc
import glob
logging.basicConfig(filename='BL_CRM_LR_Quarterly_upto20200418_2020Q2_Store_per_ID_'+str(datetime.datetime.now().date())+'.log', level=logging.INFO)
logging.info('Started')

samplerows = None

lastdate_date = datetime.date(2020,4,18) # Recent Saturday
Beginning_12_months_ago = str(lastdate_date-datetime.timedelta(days=52*7-1)) # Sunday
Beginning_18_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1)) # Sunday
Beginning_48_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*4-1)) # Sunday

lastdate_str=str(lastdate_date)
print("Active Start on: "+Beginning_12_months_ago) #>=
print("Lapsed & Store_allocation Start on: "+Beginning_18_months_ago) #>=
print("Lapsed 19-48 Start on: "+Beginning_48_months_ago) #>=

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
folder_write = '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q2/output_'+str(datetime.datetime.now().date())+'/'
try:
    os.stat(folder_write)
except:
    os.mkdir(folder_write)
    
os.getcwd()

Active Start on: 2019-04-21
Lapsed & Store_allocation Start on: 2018-10-21
Lapsed 19-48 Start on: 2016-04-24


'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q2'

In [4]:
historical_item_files=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")

daily_item_files_2019_20202=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_item_files_2019_20202_part2=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
daily_item_files_2019_20202=daily_item_files_2019_20202+daily_item_files_2019_20202_part2
del daily_item_files_2019_20202_part2

daily_item_files_2019_20202=[x for x in daily_item_files_2019_20202 if x[-4:]==".txt" and "daily" in x.lower()]

historical_item_files.sort()
historical_item_files=[x for x in historical_item_files if x.split("/MediaStormDailySalesHistory")[1][:8]>=Beginning_18_months_ago.replace("-","")]
daily_item_files_2019_20202.sort()


In [6]:
# Piece 1: item 2018-10-21 to 2019-02-09

df=pd.read_table(historical_item_files[0],dtype=str,sep="|",usecols=['transaction_dt'])
print("min_date_historical_item: "+str(df['transaction_dt'].min()))
del df
gc.collect()
# min_date_historical_item: 2018-08-05


df=pd.read_table(historical_item_files[-1],dtype=str,sep="|",usecols=['transaction_dt'])
print("max_date_historical_item: "+str(df['transaction_dt'].max()))
del df
gc.collect()
# max_date_historical_item: 2019-02-09

min_date_historical_item: 2018-10-21
max_date_historical_item: 2019-02-09


0

In [7]:
# Piece 2: item 2019-02-10 to 2020-04-18

daily_item_files_2019_20202=[x for x in daily_item_files_2019_20202 if x.split("_weeks/MediaStorm_")[1][:10]>="2019-02-10"]


#QC
df=pd.read_table(daily_item_files_2019_20202[0],dtype=str,sep="|",usecols=['transaction_dt'])
print("min_date_daily_item: "+str(df['transaction_dt'].min()))
del df
gc.collect()
# min_date_daily_item: 2019-02-10


df=pd.read_table(daily_item_files_2019_20202[-1],dtype=str,sep="|",usecols=['transaction_dt'])
print("max_date_daily_item: "+str(df['transaction_dt'].max()))
del df
gc.collect()
# max_date_daily_item: 2020-01-11

min_date_daily_item: 2019-02-10
max_date_daily_item: 2020-04-18


0

In [8]:
list_all_files=historical_item_files+daily_item_files_2019_20202

print(len(list_all_files))

78


In [9]:
list_all_files

['/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181124.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181201.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181208.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181215.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181222.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHi

In [10]:
# removed 6990
list_df_transactions_18_months=[]
i_counter=0
for file in list_all_files:
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
        df=df.rename(columns={"item_transaction_amt":"sales"})
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','subclass_transaction_amt'])
        df=df.rename(columns={"subclass_transaction_amt":"sales"})
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df[df['location_id']!="6990"]
    df['sales']=df['sales'].astype(float)
    
    df_sales=df.groupby(['customer_id_hashed','location_id'])['sales'].sum().to_frame().reset_index()
    
    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['customer_id_hashed','location_id'])['trans'].sum().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed",'location_id'],how="outer")

    list_df_transactions_18_months.append(df)
    i_counter+=1
    if i_counter%10==1:
        print(i_counter,datetime.datetime.now())

1 2020-04-24 11:52:48.918190
11 2020-04-24 12:17:46.815766
21 2020-04-24 12:30:01.620873
31 2020-04-24 12:41:21.859577
41 2020-04-24 12:53:30.070191
51 2020-04-24 13:04:56.076491
61 2020-04-24 13:20:44.608462
71 2020-04-24 13:32:22.455544


In [11]:
df_sales_by_id_store_18=pd.concat(list_df_transactions_18_months)
del list_df_transactions_18_months
gc.collect()

0

In [12]:
df_sales_by_id_store_18.shape

(104672457, 4)

In [13]:
df_sales_by_id_store_18.head(2)

,customer_id_hashed,location_id,sales,trans
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,4061,42.35,1
1,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,1528,22.20,1


In [14]:
df_sales_by_id_store_18.to_csv(folder_write+"/BL_trans_by_id_store_detail_JL_20181021_20200418_JL_"+str(datetime.datetime.now())+".csv",index=False)


In [15]:
print(df_sales_by_id_store_18.shape)
df_sales_by_id_store_18=df_sales_by_id_store_18.groupby(['customer_id_hashed','location_id'])['sales','trans'].sum().reset_index()
df_sales_by_id_store_18=df_sales_by_id_store_18.sort_values(["customer_id_hashed","trans","sales"],ascending=[True,False,False])

df_sales_by_id_store_18.to_csv(folder_write+"/BL_trans_by_id_store_agg_JL_20181021_20200418_JL_"+str(datetime.datetime.now())+".csv",index=False)


(104672457, 4)


In [16]:
df_sales_by_id_store_18=df_sales_by_id_store_18.drop_duplicates("customer_id_hashed")
print(df_sales_by_id_store_18.shape)
df_sales_by_id_store_18.to_csv(folder_write+"/BL_allocated_store_by_id_shoppers_18_months_JL_20181021_20200418_JL_"+str(datetime.datetime.now())+".csv",index=False)


(22744579, 4)


In [17]:
df_sales_by_id_store_18.head(4)

,customer_id_hashed,location_id,sales,trans
0,000000ebcf6c6a2f4302291cc9babb0760208fc683b3b5...,1606,3.8,1
1,00000135f48c68690ad3d5fc9ada41bb5cd687452007e8...,1292,57.0,1
4,000001dadc0265bf9d250566d74e0006323f18b5826641...,4682,85.8,3
5,000003066bafc6b19c55d3be5fc646a253c659e9e6baea...,5116,164.4,2
